Clasificación

In [ ]:
import numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import  train_test_split

In [ ]:
df_original =pd.read_csv('https://raw.githubusercontent.com/cocodibuja/mineria-de-texto/main/unidad-07/dataset.csv')
df_original = df_original.loc[:, ~df_original.columns.str.contains('^Unnamed')]
#df_test= df[df.etiqueta.isna()]
#df= df[~df.etiqueta.isna()]

In [ ]:
pd.value_counts(df_original['etiqueta'])

salud      818
navidad    777
vacas      329
Name: etiqueta, dtype: int64

## Balancear a mano

In [ ]:
df = pd.concat([
df_original[df_original['etiqueta']=='salud'][0:400],
df_original[df_original['etiqueta']=='navidad'][0:400],
df_original[df_original['etiqueta']=='vacas']
              ]).reset_index(drop=True)


In [ ]:
df

,Text,etiqueta
0,Dicen que los hombres piensan en el sexo cada ...,salud
1,❓ ¿Qué es la papilomatosis respiratoria?. Enté...,salud
2,CCOO denuncia la falta de test COVID en reside...,salud
3,"¡Sin complicaciones, con excelentes profesiona...",salud
4,#Salud | Cuatro hábitos que pueden causar diab...,salud
...,...,...
1124,RT @DescubreTuIsla: En CASINO METRO siempre ha...,vacas
1125,De dónde sacan la energía??? De dónde?? #vacac...,vacas
1126,un hermoso día de playa y sol 🌞 💕 \n\n#vacacio...,vacas
1127,Excelente plan para las vacaciones!!\n#vacacio...,vacas


### Asignamos NA AL 20% de cada tipo de etiqueta

In [ ]:
index_salud = df[df['etiqueta']=='salud'].index.tolist()
index_navidad = df[df['etiqueta']=='navidad'].index.tolist()
index_vacas = df[df['etiqueta']=='vacas'].index.tolist()

In [ ]:
import random
def lista_shuffle(list_index,porcentaje=20):
  '''
    retorna una lista de items de largo igual al porcentaje ingresado
    porcentaje de 0 a 100
  '''
  random.shuffle(list_index)
  N = len(list_index)
  if 0<porcentaje<100:
    N = round(len(list_index)*porcentaje/100)
  return list_index[0:N]


In [ ]:
lista_index_na = lista_shuffle(index_salud)+lista_shuffle(index_vacas)+lista_shuffle(index_navidad)

In [ ]:
df.loc[lista_index_na]

,Text,etiqueta
173,RT @YOSEquehacer2_: Finanzas con sentido común...,salud
169,‼️Estos son los mejores materiales 🌿 #naturale...,salud
136,Este último Lunes del año queremos recordarte ...,salud
18,RT @CaracolRadio: #Salud | Un experto habló s...,salud
242,RT @JCSanchezR3: Felicidades para mi muy queri...,salud
...,...,...
559,Venezuela no ae arregló…. #navidad #santa #dos...,navidad
619,RT @Pontifex_es: Esto es lo que podemos pedir ...,navidad
589,Decomisan 225 kilogramos de pirotecnia durante...,navidad
744,RT @Trust_Spain: Si existiera la #Navidad en D...,navidad


In [ ]:
df.loc[lista_index_na,'etiqueta'] = np.nan

In [ ]:
df

,Text,etiqueta
0,Dicen que los hombres piensan en el sexo cada ...,salud
1,❓ ¿Qué es la papilomatosis respiratoria?. Enté...,NaN
2,CCOO denuncia la falta de test COVID en reside...,salud
3,"¡Sin complicaciones, con excelentes profesiona...",salud
4,#Salud | Cuatro hábitos que pueden causar diab...,salud
...,...,...
1124,RT @DescubreTuIsla: En CASINO METRO siempre ha...,NaN
1125,De dónde sacan la energía??? De dónde?? #vacac...,vacas
1126,un hermoso día de playa y sol 🌞 💕 \n\n#vacacio...,vacas
1127,Excelente plan para las vacaciones!!\n#vacacio...,vacas


In [ ]:
df_test= df[df.etiqueta.isna()]
df= df[~df.etiqueta.isna()]

In [ ]:
df_test.shape # (226, 2)
df.shape # (903, 2)

(903, 2)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('spanish'))
from nltk.stem import PorterStemmer 
default_stemmer = PorterStemmer()

default_stopwords = stopwords.words('spanish') 
def clean_text(text, ):

    def tokenize_text(text):
        return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

    def remove_special_characters(text, characters=string.punctuation.replace('-', '')):
        tokens = tokenize_text(text)
        pattern = re.compile('[{}]'.format(re.escape(characters)))
        return ' '.join(filter(None, [pattern.sub('', t) for t in tokens]))

    def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

    def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

    text = text.strip(' ')
    text = text.lower()
    text = stem_text(text) 
    text = remove_special_characters(text) 
    text = remove_stopwords(text)

    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Corremos la función

message_limpio = []
for i in df['Text']:
  message_limpio.append(clean_text(str(i)))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df.etiqueta) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df['Message'] = [str (item) for item in message_limpio]

df = df.dropna()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.groupby('etiqueta').count()

,Text,label,Message
etiqueta,,,
navidad,320,320,320
salud,320,320,320
vacas,263,263,263


In [ ]:
df

,Text,etiqueta,label,Message
0,Dicen que los hombres piensan en el sexo cada ...,salud,1,dicen hombr piensan sexo cada 8 segundo mujer ...
2,CCOO denuncia la falta de test COVID en reside...,salud,1,ccoo denuncia falta test covid residencia tra ...
3,"¡Sin complicaciones, con excelentes profesiona...",salud,1,¡sin complicacion excelent profesional tratan ...
4,#Salud | Cuatro hábitos que pueden causar diab...,salud,1,salud cuatro hábito pueden causar diabet http ...
5,RT @AECID_es: La #salud es uno de los ejes pri...,salud,1,rt aecid salud eje principal trabajo aecid equ...
...,...,...,...,...
1122,Parejita de vacaciones en la nieve! \n\n#lucia...,vacas,2,parejita vacacion niev lucianodalessandro mari...
1123,Y salieron corriendo de la tormenta de nieve! ...,vacas,2,salieron corriendo tormenta niev 😂 lucianodale...
1125,De dónde sacan la energía??? De dónde?? #vacac...,vacas,2,dónde sacan energía dónde vacacion http tcouri...
1126,un hermoso día de playa y sol 🌞 💕 \n\n#vacacio...,vacas,2,hermoso día playa sol 🌞 💕 vacacion familia 🥰 h...


In [ ]:
import numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import  train_test_split
import glob
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(
    df['Message'], df['label'], test_size = 0.20, stratify=df['label'], random_state = 12)


#Entreno el modelo
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

model.fit(X_train, y_train)


y_pred = model.predict(X_test) 

#Plot de la matriz de confusión 
print(accuracy_score(y_test, y_pred))

0.856353591160221


In [ ]:
from sklearn.metrics import confusion_matrix as cm
import pandas as pd

confusion_matrix=cm(y_test, y_pred)

list1 = ["Actual navidad", "Actual salud", "Actual vacas"]
list2 = ["Predicted navidad", "Predicted salud", "Predicted vacas"]
pd.DataFrame(confusion_matrix, list1,list2)


,Predicted navidad,Predicted salud,Predicted vacas
Actual navidad,56,5,3
Actual salud,5,59,0
Actual vacas,8,5,40


In [ ]:
#Aplicamos mismo preprocesamiento que al conjunto de Train!!!


message_limpio_test = []
for i in df_test['Text']:
  message_limpio_test.append(clean_text(str(i)))


df_test['Message'] = [str (item) for item in message_limpio_test]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [ ]:
#Sumo los valores predichos por el modelo en la columna Sentiment
y_pred_sen = model.predict(df_test['Message']) 
df_test['label'] = y_pred_sen

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df_test['label'] = le.inverse_transform(df_test['label'] )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_test

,Text,etiqueta,Message,label
1,❓ ¿Qué es la papilomatosis respiratoria?. Enté...,NaN,❓ ¿qué papilomatosi respiratoria entérat notim...,salud
8,RT @HPUPaquitoGzle1: Los trabajadores de la Sa...,NaN,rt hpupaquitogzle1 trabajador salud cubano hpu...,salud
11,@DaniAmbrosino @rolandogps @AmericaTV @rodrigo...,NaN,daniambrosino rolandogp americatv rodrigosalem...,salud
18,RT @CaracolRadio: #Salud | Un experto habló s...,NaN,rt caracolradio salud experto habló sobr impor...,salud
24,He tenido que cambiar 4 veces de Comunidad Aut...,NaN,cambiar 4 vece comunidad autónoma siempr misma...,navidad
...,...,...,...,...
1116,RT @VeridicoPuebla: #Vacaciones\n#Puebla\n#Via...,NaN,rt veridicopuebla vacacion puebla viaj video m...,vacas
1119,Decía mi tobillo que mejor ahí la dejara. #lec...,NaN,decía tobillo mejor ahí dejara lectorasquecorr...,vacas
1120,RT @alirio1504: Perdoname por interrumpir tus ...,NaN,rt alirio1504 perdonam interrumpir placentera ...,navidad
1124,RT @DescubreTuIsla: En CASINO METRO siempre ha...,NaN,rt descubretuisla casino metro siempr diversió...,vacas


In [ ]:
df_test.loc[:,['Text', 'label']][190:200]

,Text,label
941,"☺️Me gusta como me miras, como planeando un vi...",vacas
943,@Profeco anunció que en el caso de retraso o ...,salud
944,Seguimos 🌳 #vacaciones #queenstown https://t.c...,vacas
948,RT @Carne_Joven_MAD: 🎭 ¿Estás aprovechando las...,vacas
951,Agendate Brinda: feria de vinos en enero en La...,vacas
953,Uno vuelve dónde fue feliz 🥺 nos regresamos a ...,vacas
956,Chaquetas Ferrara para estas salidas sin sufri...,salud
961,RT @safettimexico: Bib GRAVEL con bolsillos la...,vacas
963,Bib GRAVEL con bolsillos laterales y trasero p...,vacas
969,"#vacaciones \nSolo se reciben Memes, regalos y...",navidad
